In [2]:
from src.text_utils import custom_pos, trennbare_workaround, google_translate

text = """
Ich stehe jeden Morgen um sechs Uhr auf.
Wir kaufen am Samstag im Supermarkt ein.
Kannst du mich heute Abend anrufen?
Kommst du mit ins Kino?
"""

dest_language='en'
out_df = custom_pos(text)
out_df = trennbare_workaround(out_df, verbose=False)

only_important = out_df['norm_token'] != ""

filt_df = out_df[only_important]

#group by token and normalized token and bring all sentences
de_df = filt_df.groupby(['token', 'norm_token'], as_index=False)['sentence'].apply(list)
#Minimum position of the normalized token
tok_pos_df = filt_df.groupby('norm_token').min()['position']
#create dataset in german
de_df = de_df.merge(tok_pos_df, on='norm_token').sort_values(by='position').drop(columns='position')

tokens_to_translate = de_df['norm_token'].values
sentences_to_translate = de_df['sentence'].values

de_df['token_target'] = google_translate(de_df['norm_token'].to_list())
de_df['sentence_target'] = [google_translate(sents) for sents in de_df['sentence']]


#just fixing the final dataset
renaming = {
    'token': 'token_raw', 
    'norm_token': 'token_de', 
    'sentence': 'sentence_de', 
    'token_target': f'token_{dest_language}',
    'sentence_target': f'sentence_{dest_language}'

}
de_df.rename(columns=renaming)


Translating to en: 100%|██████████| 1/1 [00:00<00:00,  6.30it/s]


,token_raw,token_de,sentence_de,token_en,sentence_en
15,stehe,aufstehen,[Ich stehe jeden Morgen um sechs Uhr auf.],stand up,[I get up at six o'clock every morning.]
13,kaufen,einkaufen,[Wir kaufen am Samstag im Supermarkt ein.],shopping,[We shop at the supermarket on Saturday.]
7,Samstag,der Samstag,[Wir kaufen am Samstag im Supermarkt ein.],the Saturday,[We shop at the supermarket on Saturday.]
8,Supermarkt,der Supermarkt,[Wir kaufen am Samstag im Supermarkt ein.],the supermarket,[We shop at the supermarket on Saturday.]
11,ein,ein,[Wir kaufen am Samstag im Supermarkt ein.],a,[We shop at the supermarket on Saturday.]
3,Kannst,Kannst,[Kannst du mich heute Abend anrufen?],Can,[Can you call me tonight?]
12,heute,heute,[Kannst du mich heute Abend anrufen?],today,[Can you call me tonight?]
0,Abend,der Abend,[Kannst du mich heute Abend anrufen?],the evening,[Can you call me tonight?]
10,anrufen,anrufen,[Kannst du mich heute Abend anrufen?],call,[Can you call me tonight?]
1,Bitte,bitte,[Bitte mach das Fenster auf.],please,[Please open the window.]


In [19]:
import googletrans

new_dict = {}
for key, value in googletrans.LANGUAGES.items():
    new_dict[value] = key

new_dict